In [1]:
%load_ext memory_profiler

In [18]:
import pyarrow as pa
from datetime import datetime

In [12]:
with pa.OSFile('data/example_01.dat', 'wb') as f:
    f.write(b'some example data' * 30 * 1024 * 1024)

In [13]:
%ls -lh data 

total 511M
-rw-r--r-- 1 jovyan users 510M Aug 10 13:11 example_01.dat


In [3]:
%%memit
mmap = pa.memory_map('data/example_01.dat')

peak memory: 66.13 MiB, increment: 0.51 MiB


In [13]:
mmap.seek(0)

0

In [14]:
buf = mmap.read_buffer(10)

In [16]:
buf.size

10

In [17]:
my_schema = pa.schema([('integer', pa.int32()),
                       ('text', pa.string()),
                       ('binary', pa.binary(10)),
                       ('Time', pa.timestamp('ms'))])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  after removing the cwd from sys.path.


In [19]:
data = [
    pa.array([1, 2, 3, 4]),
    pa.array(['foo', 'bar', 'baz', None]),
    pa.array([0b0100, None, 0b0100, 0b0100]),
    pa.array((datetime(2019,8,i) for i in range(1, 5)))
]

A **Record Batch** in Apache Arrow is a collection of equal-length array instances.

In [56]:
pa.RecordBatch?

Init signature: pa.RecordBatch(self, /, *args, **kwargs)
Docstring:     
RecordBatch()

Batch of rows of columns of equal length

Warning
-------
Do not call this class's constructor directly, use one of the
``RecordBatch.from_*`` functions instead.
File:           /opt/conda/lib/python3.7/site-packages/pyarrow/lib.cpython-37m-x86_64-linux-gnu.so
Type:           type
Subclasses:     


In [54]:
batch1 = pa.RecordBatch.from_arrays(data, my_schema)

with pa.OSFile('data/example_02.dat', 'wb') as f:
    writer = pa.RecordBatchStreamWriter(f, batch1.schema)

    for i in range(1024 * 1024):
        writer.write_batch(batch1)

In [55]:
%%memit
mmap = pa.memory_map('data/example_02.dat')

peak memory: 2309.64 MiB, increment: 0.00 MiB


In [37]:
mmap.seek(0)
mmap.read(100)

b'\x1c\x01\x00\x00\x10\x00\x00\x00\x00\x00\n\x00\x0c\x00\x06\x00\x05\x00\x08\x00\n\x00\x00\x00\x00\x01\x03\x00\x0c\x00\x00\x00\x08\x00\x08\x00\x00\x00\x04\x00\x08\x00\x00\x00\x04\x00\x00\x00\x04\x00\x00\x00\xb4\x00\x00\x00t\x00\x00\x00<\x00\x00\x00\x04\x00\x00\x00l\xff\xff\xff\x00\x00\x01\n \x00\x00\x00\x14\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x06\x00\x08\x00\x06\x00'

In [51]:
%%memit
reader = pa.RecordBatchStreamReader('data/example_02.dat')

peak memory: 2613.88 MiB, increment: 0.01 MiB


In [52]:
df = reader.read_pandas()

In [53]:
df.head()

,integer,text,binary,Time
0,1,foo,b'\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00',1863-08-31 12:02:14.688112640
1,0,bar,None,1866-05-27 12:02:14.688112640
2,2,baz,b'\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00',1869-02-20 12:02:14.688112640
3,0,None,b'\x00\x00\x00\xe0A\xec\x02\x8f\x05\x00',1871-11-17 12:02:14.688112640
4,1,foo,b'\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00',1863-08-31 12:02:14.688112640
